In [15]:
import numpy as np
import pandas as pd

In [16]:
cuts=[0,.42,.47,.48,.49,.5,.51,.52,.53,.55,1]
cuts

[0, 0.42, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.55, 1]

This is our regular imports and then "cuts" sets up what will be the categories that we put states into by voting percent.  That is "binning".  So for example above, everything from .5 to .51 is one category.

In Cell below, adjust it to use the file on the location in your computer.

The **head()** function lets us see part of a dataset.  "raw" is a dataframe.  Basically what happens below is we trim out extra columns and "pivot" the Republican and Democrat row for each state into a single row.

In [18]:
raw = pd.read_csv("/users/pa/Downloads/1976-2016-president.csv")
raw.head(8)

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes
0,1976,Alabama,AL,1,63,41,US President,"Carter, Jimmy",democrat,False,659170,1182850,20171015,NaN
1,1976,Alabama,AL,1,63,41,US President,"Ford, Gerald",republican,False,504070,1182850,20171015,NaN
2,1976,Alabama,AL,1,63,41,US President,"Maddox, Lester",american independent party,False,9198,1182850,20171015,NaN
3,1976,Alabama,AL,1,63,41,US President,"Bubar, Benjamin ""Ben""",prohibition,False,6669,1182850,20171015,NaN
4,1976,Alabama,AL,1,63,41,US President,"Hall, Gus",communist party use,False,1954,1182850,20171015,NaN
5,1976,Alabama,AL,1,63,41,US President,"Macbride, Roger",libertarian,False,1481,1182850,20171015,NaN
6,1976,Alabama,AL,1,63,41,US President,NaN,NaN,True,308,1182850,20171015,NaN
7,1976,Alaska,AK,2,94,81,US President,"Ford, Gerald",republican,False,71555,123574,20171015,NaN


Because tableau can't do anything useful with all those minor candidate people, and we only want one year... we just throw out everything else using the "query" operator.  That doesn't actually change hte "raw" dataframe, it creates a new dataframe, which I assign to name "trumpraw"

In [19]:
trumpraw = raw.query('year==2016 and (party=="democrat" or party=="republican")')
trumpraw.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes
3394,2016,Alabama,AL,1,63,41,US President,"Trump, Donald J.",republican,False,1318255,2123372,20171015,NaN
3395,2016,Alabama,AL,1,63,41,US President,"Clinton, Hillary",democrat,False,729547,2123372,20171015,NaN
3399,2016,Alaska,AK,2,94,81,US President,"Trump, Donald J.",republican,False,163387,318608,20171015,NaN
3400,2016,Alaska,AK,2,94,81,US President,"Clinton, Hillary",democrat,False,116454,318608,20171015,NaN
3406,2016,Arizona,AZ,4,86,61,US President,"Trump, Donald J.",republican,False,1252401,2573165,20171015,NaN


Cut it down to the needed columns, again, this makes a new dataframe. The double brackets is strange-looking syntax.  There is a list which has only one member, and that member is a list of the columns we want.  I have to read up more on this "reshaping" operation.  

In [20]:
needed=trumpraw[['state', 'candidate', 'candidatevotes', 'totalvotes']]
needed.head()

,state,candidate,candidatevotes,totalvotes
3394,Alabama,"Trump, Donald J.",1318255,2123372
3395,Alabama,"Clinton, Hillary",729547,2123372
3399,Alaska,"Trump, Donald J.",163387,318608
3400,Alaska,"Clinton, Hillary",116454,318608
3406,Arizona,"Trump, Donald J.",1252401,2573165


We're going to want to combine Trump and Clinton on one line for each state to graph in Tableua.  Now we want to check and make sure there are only two rows per state . . . Darn it, there are some that have more: 

In [9]:
defs = neeed.groupby(['state']).count()
defs.query("totalvotes != 2")

,candidate,candidatevotes,totalvotes
state,,,
Arizona,2,3,3
Maryland,4,4,4


Check the details:

In [11]:
needed.query("state == 'Maryland' or state == 'Arizona'")

,state,candidate,candidatevotes,totalvotes
3406,Arizona,"Trump, Donald J.",1252401,2573165
3407,Arizona,"Clinton, Hillary",1161167,2573165
3413,Arizona,NaN,42,2573165
3534,Maryland,"Clinton, Hillary",1677928,2781446
3535,Maryland,"Trump, Donald J.",943169,2781446
3541,Maryland,"Trump, Donald J.",259,2781446
3542,Maryland,"Clinton, Hillary",78,2781446


Maybe it would be best to total them?  But they mostly look inconsequential.  So we'll delete them.  First we get a list of the linesto delete into a variable named 'delme':

In [12]:
dels = needed.query('candidatevotes<360')
delme=dels.index.tolist()
delme

[3413, 3541, 3542]

Delete them with "drop" command, and check there are 102 rows (50 states + 1 district) * 2 parties

In [13]:
good=needed.drop(delme)
len(good)


102

In [233]:
good.head()

,state,candidate,candidatevotes,totalvotes
3394,Alabama,"Trump, Donald J.",1318255,2123372
3395,Alabama,"Clinton, Hillary",729547,2123372
3399,Alaska,"Trump, Donald J.",163387,318608
3400,Alaska,"Clinton, Hillary",116454,318608
3406,Arizona,"Trump, Donald J.",1252401,2573165


"pivoting" turns multiple rows into one, moving the value of "pivoted" for each group ino one row.  You can see the result below from the **head()** command:

In [238]:
oneline=good.pivot(index='state', columns='candidate', values=['candidatevotes','totalvotes'])
oneline.head()

candidatevotes                        totalvotes                 
candidate  Clinton, Hillary Trump, Donald J. Clinton, Hillary Trump, Donald J.
state                                                                         
Alabama              729547          1318255          2123372          2123372
Alaska               116454           163387           318608           318608
Arizona             1161167          1252401          2573165          2573165
Arkansas             380494           684872          1130635          1130635
California          8753788          4483810         14181595         14181595

In [240]:
oneline.columns = ['A','B','C','D']

In [243]:
tograph=oneline.drop('D',1)
tograph.head()

,A,B,C
state,,,
Alabama,729547,1318255,2123372
Alaska,116454,163387,318608
Arizona,1161167,1252401,2573165
Arkansas,380494,684872,1130635
California,8753788,4483810,14181595


In [244]:
tograph.columns =[ 'Clinton', 'Trump','Total']

In [246]:
tograph['dempct'] = tograph['Clinton']/tograph["Total"]
tograph.head()

,Clinton,Trump,Total,dempct
state,,,,
Alabama,729547,1318255,2123372,0.343579
Alaska,116454,163387,318608,0.365509
Arizona,1161167,1252401,2573165,0.451260
Arkansas,380494,684872,1130635,0.336531
California,8753788,4483810,14181595,0.617264


In [249]:
cuts

[0, 0.42, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.55, 1]

In [259]:
cats=pd.cut(tograph['dempct'],bins=cuts, labels=list('9876543210'))
cats.head()

state
Alabama       9
Alaska        9
Arizona       8
Arkansas      9
California    0
Name: dempct, dtype: category
Categories (10, object): [9 < 8 < 7 < 6 ... 3 < 2 < 1 < 0]

In [261]:
tograph['cat']=cats
tograph.head()

,Clinton,Trump,Total,dempct,cat
state,,,,,
Alabama,729547,1318255,2123372,0.343579,9
Alaska,116454,163387,318608,0.365509,9
Arizona,1161167,1252401,2573165,0.451260,8
Arkansas,380494,684872,1130635,0.336531,9
California,8753788,4483810,14181595,0.617264,0
